In [1]:
import yaml
import pandas as pd
import geopandas as gpd
import geohash2
from shapely import geometry
from pyproj import Transformer
from polygeohasher import polygeohasher # https://www.geospatialworld.net/blogs/polygeohasher-an-optimized-way-to-create-geohashes/



## Load Config

In [2]:
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
# Set Configs
paths: dict() = config['paths']
crs: dict() = config['crs']


In [4]:
# Coord System Transformer
transformer_gps_to_osmeters = Transformer.from_crs(crs['latlon_gps'], crs['proj_os32'])

## Functions

In [5]:
def geohash_to_polygon(ghash: str) -> geometry.Polygon:

    lat_centroid, lng_centroid, lat_offset, lng_offset = geohash2.decode_exactly(ghash)

    corner_1 = transformer_gps_to_osmeters.transform(lat_centroid - lat_offset, lng_centroid - lng_offset)
    corner_2 = transformer_gps_to_osmeters.transform(lat_centroid + lat_offset, lng_centroid - lng_offset)
    corner_3 = transformer_gps_to_osmeters.transform(lat_centroid + lat_offset, lng_centroid + lng_offset)
    corner_4 = transformer_gps_to_osmeters.transform(lat_centroid - lat_offset, lng_centroid + lng_offset)

    polygon = geometry.Polygon([corner_1, corner_2, corner_3, corner_4, corner_1])

    return polygon

## Main

In [6]:
# Load Polygon to GeoDataFrame
gdf_Polygon: gpd.GeoDataFrame = gpd.read_file(paths['area_polygon'], crs=crs['meters'])

In [7]:
# Filter by Region
gdf_Polygon = gdf_Polygon[gdf_Polygon['Region'] == config['region']]

In [8]:
# Convert CRS to GPS
gdf_Polygon_GPS = gdf_Polygon.to_crs(crs['latlon_gps'])

In [9]:
# Melt to single polygon
union_geom = gdf_Polygon_GPS['geometry'].unary_union

In [10]:
# Create an interim Pandas dataframe
df_Union = pd.DataFrame({'id':'0', 'geometry': union_geom}, index=[0])

In [11]:
# Create GeoDataFrame with unionized polygon
gdf_Union = gpd.GeoDataFrame(df_Union, crs=crs['latlon_gps'])

In [12]:
# Get 
gdf_poly_hashed = polygeohasher.create_geohash_list(gdf=gdf_Union, geohash_level=config['geohash_precision'], inner=False)

In [13]:
# Use Set to remove any duplicates
hash_set = set(gdf_poly_hashed['geohash_list'].to_list()[0])

In [14]:
# Create dictionary where Key = GeoHash, Value = Polygon shape
hash_dict = [{'GeoHash': n, 'geometry': geohash_to_polygon(n)} for n in hash_set]

In [15]:
# Create GeoDataFrame
gdf_hash_dict = gpd.GeoDataFrame(data=hash_dict, crs=crs['meters'])

In [16]:
# Set GeoHash as index
gdf_hash_dict.set_index('GeoHash', inplace=True)

In [17]:
# Export file
gdf_hash_dict.to_feather(path=paths['save_as']+'.feather', index=True)

In [18]:
print('Completed')

Completed
